In [ ]:
#1.
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from skimage import exposure

# Thư mục chứa ảnh đầu vào
input_dir = 'Exercise'
output_dir = 'Output'
os.makedirs(output_dir, exist_ok=True)

# Các hàm biến đổi ảnh grayscale
def inverse_transform(img):
    return 255 - img

def gamma_correction(img, gamma=0.5):
    norm_img = img / 255.0
    gamma_corrected = np.power(norm_img, gamma) * 255
    return np.clip(gamma_corrected, 0, 255).astype(np.uint8)

def log_transform(img):
    c = 255 / np.log(1 + np.max(img))
    log_image = c * np.log(1 + img)
    return np.clip(log_image, 0, 255).astype(np.uint8)

def histogram_equalization(img):
    return cv2.equalizeHist(img)

def contrast_stretching(img):
    a = np.min(img)
    b = np.max(img)
    stretched = (img - a) * (255 / (b - a))
    return np.clip(stretched, 0, 255).astype(np.uint8)

# Tạo menu lựa chọn
print("Chọn phương pháp biến đổi ảnh:")
print("I - Image Inverse")
print("G - Gamma Correction")
print("L - Log Transformation")
print("H - Histogram Equalization")
print("C - Contrast Stretching")

key = input("Nhập phím I, G, L, H, C: ").upper()

# Ánh xạ phím bấm đến hàm tương ứng
functions = {
    'I': ('inverse', inverse_transform),
    'G': ('gamma', gamma_correction),
    'L': ('log', log_transform),
    'H': ('hist_eq', histogram_equalization),
    'C': ('contrast', contrast_stretching)
}

if key not in functions:
    print("Phím không hợp lệ.")
    exit()

label, func = functions[key]

# Duyệt ảnh trong thư mục và áp dụng biến đổi
for file in os.listdir(input_dir):
    if file.endswith(('.jpg', '.png', '.bmp')):
        path = os.path.join(input_dir, file)
        img = Image.open(path).convert('L')
        img_np = np.array(img)

        transformed = func(img_np)

        output_path = os.path.join(output_dir, f"{os.path.splitext(file)[0]}_{label}.png")
        Image.fromarray(transformed).save(output_path)

        # Hiển thị kết quả
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        plt.title("Original")
        plt.imshow(img_np, cmap='gray')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title(f"Transformed ({label})")
        plt.imshow(transformed, cmap='gray')
        plt.axis('off')
        plt.tight_layout()
        plt.show()


In [ ]:
#3.
import os
import random
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Thư mục ảnh đầu vào
input_dir = 'Exercise'
output_dir = 'Output_Q3'
os.makedirs(output_dir, exist_ok=True)

# Các hàm xử lý ảnh grayscale (từ câu 1)
def inverse_transform(img): return 255 - img

def gamma_correction(img, gamma=0.5):
    return np.clip(np.power(img / 255.0, gamma) * 255, 0, 255).astype(np.uint8)

def log_transform(img):
    c = 255 / np.log(1 + np.max(img))
    return np.clip(c * np.log(1 + img), 0, 255).astype(np.uint8)

def histogram_equalization(img): return cv2.equalizeHist(img)

def contrast_stretching(img):
    a, b = np.min(img), np.max(img)
    return np.clip((img - a) * (255 / (b - a)), 0, 255).astype(np.uint8)

# Danh sách các phép biến đổi
transforms = [
    ('inverse', inverse_transform),
    ('gamma', gamma_correction),
    ('log', log_transform),
    ('hist_eq', histogram_equalization),
    ('contrast', contrast_stretching)
]

# Tất cả hoán vị RGB có thể
rgb_permutations = [
    (0, 1, 2), (0, 2, 1), (1, 0, 2),
    (1, 2, 0), (2, 0, 1), (2, 1, 0)
]

# Duyệt ảnh
for file in os.listdir(input_dir):
    if file.lower().endswith(('.jpg', '.png', '.bmp')):
        path = os.path.join(input_dir, file)
        img = cv2.imread(path)

        # Thay đổi thứ tự RGB ngẫu nhiên
        perm = random.choice(rgb_permutations)
        img_rgb_changed = img[:, :, list(perm)]

        # Chuyển sang ảnh xám để áp dụng biến đổi
        gray_img = cv2.cvtColor(img_rgb_changed, cv2.COLOR_BGR2GRAY)

        # Chọn biến đổi ngẫu nhiên từ câu 1
        label, transform_func = random.choice(transforms)
        processed = transform_func(gray_img)

        # Lưu và hiển thị kết quả
        output_path = os.path.join(output_dir, f"{os.path.splitext(file)[0]}_{label}_perm{perm}.png")
        cv2.imwrite(output_path, processed)

        # Hiển thị
        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        plt.title("RGB Changed")
        plt.imshow(cv2.cvtColor(img_rgb_changed, cv2.COLOR_BGR2RGB))
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title(f"Transformed ({label})")
        plt.imshow(processed, cmap='gray')
        plt.axis('off')
        plt.tight_layout()
        plt.show()


In [ ]:
#4.
import os
import cv2
import numpy as np
import random
from scipy import ndimage
import matplotlib.pyplot as plt

# Tạo thư mục đầu ra
input_dir = 'Exercise'
output_dir = 'Output_Q4'
os.makedirs(output_dir, exist_ok=True)

# Hàm Fourier transform
def fourier_transform(img):
    f = np.fft.fft2(img)
    fshift = np.fft.fftshift(f)
    magnitude = 20 * np.log(np.abs(fshift) + 1)
    return np.clip(magnitude, 0, 255).astype(np.uint8)

# Butterworth Lowpass
def butterworth_lowpass(img, cutoff=30, order=2):
    rows, cols = img.shape
    u = np.array(range(rows)) - rows // 2
    v = np.array(range(cols)) - cols // 2
    U, V = np.meshgrid(u, v, indexing='ij')
    D = np.sqrt(U**2 + V**2)
    H = 1 / (1 + (D / cutoff)**(2 * order))
    F = np.fft.fftshift(np.fft.fft2(img))
    G = H * F
    result = np.abs(np.fft.ifft2(np.fft.ifftshift(G)))
    return np.clip(result, 0, 255).astype(np.uint8)

# Butterworth Highpass
def butterworth_highpass(img, cutoff=30, order=2):
    rows, cols = img.shape
    u = np.array(range(rows)) - rows // 2
    v = np.array(range(cols)) - cols // 2
    U, V = np.meshgrid(u, v, indexing='ij')
    D = np.sqrt(U**2 + V**2)
    H = 1 / (1 + (cutoff / (D + 1e-5))**(2 * order))
    F = np.fft.fftshift(np.fft.fft2(img))
    G = H * F
    result = np.abs(np.fft.ifft2(np.fft.ifftshift(G)))
    return np.clip(result, 0, 255).astype(np.uint8)

# Max và Min filter
def apply_min_filter(img, size=3):
    return ndimage.minimum_filter(img, size=size)

def apply_max_filter(img, size=3):
    return ndimage.maximum_filter(img, size=size)

# Danh sách các phép biến đổi
transform_options = [
    ('fourier', fourier_transform),
    ('butter_low', butterworth_lowpass),
    ('butter_high', butterworth_highpass)
]

# Tất cả hoán vị RGB có thể
rgb_permutations = [
    (0, 1, 2), (0, 2, 1), (1, 0, 2),
    (1, 2, 0), (2, 0, 1), (2, 1, 0)
]

# Duyệt qua các ảnh
for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.jpg', '.png')):
        path = os.path.join(input_dir, filename)
        img = cv2.imread(path)

        # Hoán đổi thứ tự RGB ngẫu nhiên
        perm = random.choice(rgb_permutations)
        img_permuted = img[:, :, list(perm)]

        # Chuyển sang grayscale
        gray = cv2.cvtColor(img_permuted, cv2.COLOR_BGR2GRAY)

        # Chọn phép biến đổi
        trans_label, trans_func = random.choice(transform_options)
        result = trans_func(gray)

        # Nếu là Butterworth Lowpass → Min Filter
        if trans_label == 'butter_low':
            result = apply_min_filter(result)

        # Nếu là Butterworth Highpass → Max Filter
        if trans_label == 'butter_high':
            result = apply_max_filter(result)

        # Lưu và hiển thị
        save_name = f"{os.path.splitext(filename)[0]}_{trans_label}_perm{perm}.png"
        save_path = os.path.join(output_dir, save_name)
        cv2.imwrite(save_path, result)

        plt.figure(figsize=(10, 4))
        plt.subplot(1, 2, 1)
        plt.title("RGB Changed")
        plt.imshow(cv2.cvtColor(img_permuted, cv2.COLOR_BGR2RGB))
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.title(f"Applied: {trans_label}")
        plt.imshow(result, cmap='gray')
        plt.axis('off')
        plt.tight_layout()
        plt.show()
